This file starts with the OpenStates bulk download file called 2017-07-01-tx-json, from https://openstates.org/downloads/. It then deletes the rows not needed (yet) for the InfluenceTX campaign finance web service and makes two big JSON files to be loaded into the InfluenceTX database. One is for Texas Senate votes on House bills and the other is for Senate votes on Senate bills.

In [1]:
import json
import shutil, os

def getContents(path):
    with open(path) as data_file:
        bill = json.load(data_file)
    if "Resolutions--Congratulatory & Honorary (I0705)" in bill['scraped_subjects']:
        return None
    bill.pop('actions', None)
    bill['votes'] = [x for x in bill['votes'] if x['motion'] == "passage"] # limited to votes on passage
    bill['votes'] = [x for x in bill['votes'] if x['chamber'] == "upper"] # only Senate votes were requested for this
    return bill


In [2]:
def getfiles(pathToDir):
    j = {}
    for root, dirs, files in os.walk(pathToDir):
        for filename in files:
            if filename != ".DS_Store":
                bill = getContents(os.path.join(pathToDir, filename))
                if bill != None:
                    j[filename] = bill
    return j

folderName = './2017-07-01-tx-json/bills/tx/85/'
senate = getfiles(os.path.join(folderName, "upper"))
house = getfiles(os.path.join(folderName, "lower"))

In [3]:
with open('../data/votes/houseBillVotes.json', 'w') as fp:
    json.dump(house, fp)

with open('../data/votes/senateBillVotes.json', 'w') as fp:
    json.dump(senate, fp)